In [1]:
import numpy as mp
import pandas as pd
import random
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
data = pd.read_csv('resume_data.csv')
data.head()

,Resume_str,description
0,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,Overview\n\nHearingLife is a national hearing ...
1,"HR SPECIALIST, US HR OPERATIONS ...",Metalcraft of Mayville\nMetalcraft of Mayville...
2,HR DIRECTOR Summary Over 2...,\nThe TSUBAKI name is synonymous with excellen...
3,HR SPECIALIST Summary Dedica...,descriptionTitle\n\n Looking for a great oppor...
4,HR MANAGER Skill Highlights ...,"Job Summary\nAt iHerb, we are on a mission to ..."


In [3]:
# Selecting columns by name
# new_df = df[['Resume_str', 'description']]
# data.drop('Unnamed: 0', axis=1, inplace=True)
data['resume_id'] = range(1, len(data) + 1)

In [4]:
data.head(20)

,Resume_str,description,resume_id
0,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,Overview\n\nHearingLife is a national hearing ...,1
1,"HR SPECIALIST, US HR OPERATIONS ...",Metalcraft of Mayville\nMetalcraft of Mayville...,2
2,HR DIRECTOR Summary Over 2...,\nThe TSUBAKI name is synonymous with excellen...,3
3,HR SPECIALIST Summary Dedica...,descriptionTitle\n\n Looking for a great oppor...,4
4,HR MANAGER Skill Highlights ...,"Job Summary\nAt iHerb, we are on a mission to ...",5
5,HR GENERALIST Summary Dedic...,"Location: Remote, CT, United States of America...",6
6,HR MANAGER Summary HUMAN RES...,Company: Gerber Collision & Glass\n\nWELCOME T...,7
7,HR MANAGER Professional Summa...,The First Assistant Store Director is actively...,8
8,HR SPECIALIST Summary Posses...,"descriptionTitle\n\n $2,000 Sign-on Bonus Guar...",9
9,HR CLERK Summary Translates ...,Instrumentation Quality Control Representative...,10


In [72]:
# new_df.to_csv('resume_data.csv', index=False)

In [73]:
# data = pd.read_csv('resume_data.csv')
# data.head()

In [5]:
data.isnull().sum()

Resume_str     30532
description        1
resume_id          0
dtype: int64

In [6]:
data.dropna(inplace=True)

In [7]:
data.duplicated().sum()

0

In [8]:
data['Resume_str']=data['Resume_str'].apply(lambda x:x.split())

In [9]:
data.head()

,Resume_str,description,resume_id
0,"[HR, ADMINISTRATOR/MARKETING, ASSOCIATE, HR, A...",Overview\n\nHearingLife is a national hearing ...,1
1,"[HR, SPECIALIST,, US, HR, OPERATIONS, Summary,...",Metalcraft of Mayville\nMetalcraft of Mayville...,2
2,"[HR, DIRECTOR, Summary, Over, 20, years, exper...",\nThe TSUBAKI name is synonymous with excellen...,3
3,"[HR, SPECIALIST, Summary, Dedicated,, Driven,,...",descriptionTitle\n\n Looking for a great oppor...,4
4,"[HR, MANAGER, Skill, Highlights, HR, SKILLS, H...","Job Summary\nAt iHerb, we are on a mission to ...",5


In [10]:
data['Resume_str']=data['Resume_str'].apply(lambda x:" ".join(x))

In [11]:
data.head()

,Resume_str,description,resume_id
0,HR ADMINISTRATOR/MARKETING ASSOCIATE HR ADMINI...,Overview\n\nHearingLife is a national hearing ...,1
1,"HR SPECIALIST, US HR OPERATIONS Summary Versat...",Metalcraft of Mayville\nMetalcraft of Mayville...,2
2,HR DIRECTOR Summary Over 20 years experience i...,\nThe TSUBAKI name is synonymous with excellen...,3
3,"HR SPECIALIST Summary Dedicated, Driven, and D...",descriptionTitle\n\n Looking for a great oppor...,4
4,HR MANAGER Skill Highlights HR SKILLS HR Depar...,"Job Summary\nAt iHerb, we are on a mission to ...",5


In [12]:
data['Resume_str'] = data['Resume_str'].apply(lambda x:x.lower())

In [13]:
data.head()

,Resume_str,description,resume_id
0,hr administrator/marketing associate hr admini...,Overview\n\nHearingLife is a national hearing ...,1
1,"hr specialist, us hr operations summary versat...",Metalcraft of Mayville\nMetalcraft of Mayville...,2
2,hr director summary over 20 years experience i...,\nThe TSUBAKI name is synonymous with excellen...,3
3,"hr specialist summary dedicated, driven, and d...",descriptionTitle\n\n Looking for a great oppor...,4
4,hr manager skill highlights hr skills hr depar...,"Job Summary\nAt iHerb, we are on a mission to ...",5


In [50]:
!pip install nltk

In [14]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [15]:
def stem(text):
    y = []
    
    for i in text.split():
        y.append(ps.stem(i))
        
    return " ".join(y)

In [16]:
data['Resume_str'] = data['Resume_str'].apply(stem)

In [17]:
vectors = cv.fit_transform(data['Resume_str']).toarray()

In [18]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 2, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [19]:
cv.get_feature_names()

['00',
 '000',
 '01',
 '02',
 '03',
 '04',
 '05',
 '06',
 '07',
 '08',
 '09',
 '0px',
 '10',
 '100',
 '1000',
 '101',
 '1099',
 '10g',
 '11',
 '110',
 '11g',
 '12',
 '120',
 '125',
 '13',
 '130',
 '14',
 '140',
 '15',
 '150',
 '1500',
 '16',
 '17',
 '18',
 '180',
 '19',
 '1975',
 '1977',
 '1978',
 '1979',
 '1980',
 '1981',
 '1982',
 '1983',
 '1984',
 '1985',
 '1986',
 '1987',
 '1988',
 '1989',
 '1990',
 '1991',
 '1992',
 '1993',
 '1994',
 '1995',
 '1996',
 '1997',
 '1998',
 '1999',
 '1m',
 '1st',
 '20',
 '200',
 '2000',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '200k',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 '2021',
 '21',
 '22',
 '23',
 '24',
 '25',
 '250',
 '26',
 '27',
 '28',
 '29',
 '2d',
 '2m',
 '2nd',
 '30',
 '300',
 '3000',
 '31',
 '32',
 '33',
 '34',
 '35',
 '350',
 '36',
 '360',
 '365',
 '37',
 '38',
 '3d',
 '3m',
 '3rd',
 '40',
 '400',
 '401',
 '401k',
 '42',
 '43',
 '44',
 '45

In [20]:
similarity = cosine_similarity(vectors)

In [21]:
similarity[1]

array([0.51158978, 1.        , 0.43088752, ..., 0.27750819, 0.42160865,
       0.28379835])

In [22]:
sorted(list(enumerate(similarity[0])),reverse=True, key=lambda x:x[1])[1:6]

[(656, 0.5771042193490526),
 (717, 0.572702783397957),
 (36, 0.5725214699576926),
 (718, 0.568510459558774),
 (640, 0.5684952921283706)]

In [23]:
def recommend(desc):
    index = data[data['description']==desc].index[0]
    distances = similarity[index]
    resume_list = sorted(list(enumerate(distances)),reverse=True, key=lambda x:x[1])[1:7]
    
    for i in resume_list:
        print(data.iloc[i[0]].resume_id)

In [24]:
text = data.loc[4, 'description']
print(text)

Job Summary
At iHerb, we are on a mission to make health and wellness accessible to everyone globally through our

products and platforms. The Infrastructure Engineering team at iHerb focuses on improving our

infrastructure, scalability, performance, reliability, and security of the systems and tools for our technology

teams. We provide a wide range of services internally to our teams. Infrastructure Engineering teams

comprises Cloud Operations (CloudOps), Site Reliability Engineering (SRE), Network Engineering

(NetEng), and Production Operations (ProdOps).

The Principal Cloud Security Engineer/Architect is developing and implementing an extensive and well

defined cloud security architecture. Ensure that our security standards are meeting and exceeding cloud

security best practices. Lead our cloud security strategy, drive remediation with cloud security tooling,

optimize cloud architecture and design, and work cross functionally to define new security capabilities.

Partner wit

In [25]:
recommend(text)

86
32
59
103
12
66


In [26]:
import pickle

In [27]:
pickle.dump(data, open('data.pkl', 'wb'))

In [28]:
new_dict = data.to_dict()

In [29]:
pickle.dump(data.to_dict(), open('data_dict.pkl', 'wb'))

In [30]:
pickle.dump(similarity, open('similarity.pkl', 'wb'))